In [1]:
! pip install lxml geopy requests pandas html5lib

In [28]:
import requests
from datetime import datetime 
from geopy.geocoders import Nominatim
import pandas as pd
from time import sleep 


In [40]:
def getWeather(lat,lon,key):
    url = "https://api.openweathermap.org/data/2.5/onecall"
    params = { "lat" : lat, "lon" : lon, "appid" : key,  "units" : "imperial",  "exclude" : "current,minutely,hourly,alerts" }
    response = requests.get(url, params = params)
    return response.json()

geolocator = Nominatim(user_agent="weather_app")
apikey = "f8dc07f223611eefcb8dc902b44a184a"

df = pd.read_html("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population", flavor="lxml")
cities = df[4]
collection = []
for c in cities.to_records()[:200]:
    city = c['City'] if c['City'].find("[") < 0 else c['City'][:c['City'].find("[")]
    address = f"{city}, {c['State[c]']}"
    print(address)
    sleep(3)
    location = geolocator.geocode(address)
    try:
        weather = getWeather(location.latitude, location.longitude, apikey)
        for w in weather['daily']:
            row = { 
                "city" : city, 
                "state" : c['State[c]'], 
                "2020census" : c['2020census'], 
                "latitude" : location.latitude, 
                "longitude" : location.longitude, 
                "timezone" : weather['timezone'],
                "date" : datetime.utcfromtimestamp(w['dt']).strftime('%Y-%m-%d'),
                "condition" : w['weather'][0]['main'],
                "description" : w['weather'][0]['description'],
                "moon_phase" : w['moon_phase'],
                "temperature" : w['temp'],
                "pressure" : w['pressure'],
                "pct_humidity" : w['humidity'],
                "dew_point" : w['dew_point'],
                "wind" : { "speed" : w['wind_speed'], "direction_deg": w['wind_deg'], "gust" : w.get('wind_gust',0) },
                "uv_index" : w['uvi'],
                "pct_clouds" : w["clouds"],
                "rainfall" : w.get("rain",0),
                "snowfall" : w.get("snow",0)
            }
            collection.append(row)
    except Exception as e:
        print(f"ERROR: {e}")



New York, New York
Los Angeles, California
Chicago, Illinois
Houston, Texas
Phoenix, Arizona
Philadelphia, Pennsylvania
San Antonio, Texas
San Diego, California
Dallas, Texas
San Jose, California
Austin, Texas
Jacksonville, Florida
Fort Worth, Texas
Columbus, Ohio
Indianapolis, Indiana
Charlotte, North Carolina
San Francisco, California
Seattle, Washington
Denver, Colorado
Washington, District of Columbia
Nashville, Tennessee
Oklahoma City, Oklahoma
El Paso, Texas
Boston, Massachusetts
Portland, Oregon
Las Vegas, Nevada
Detroit, Michigan
Memphis, Tennessee
Louisville, Kentucky
Baltimore, Maryland
Milwaukee, Wisconsin
Albuquerque, New Mexico
Tucson, Arizona
Fresno, California
Sacramento, California
Kansas City, Missouri
Mesa, Arizona
Atlanta, Georgia
Omaha, Nebraska
Colorado Springs, Colorado
Raleigh, North Carolina
Long Beach, California
Virginia Beach, Virginia
Miami, Florida
Oakland, California
Minneapolis, Minnesota
Tulsa, Oklahoma
Bakersfield, California
Wichita, Kansas
Arlington, 

In [39]:
df = pd.json_normalize(collection)

df.to_json("weather.json", orient="records")
df.to_csv("weather.csv",index=False,header=True)


In [10]:
w = getWeather(43,-76, apikey)
ts = int(1634642545)
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d'))

2021-10-19


In [35]:
x = "Buffalo[b], New York"
x[:x.find("[")] + x[x.find(","):]

'Buffalo, New York'